In [1]:
from datetime import datetime, timedelta
import scrapbook as sb
from get_secret_or_env_var import environ, getenv
from flowclient import (
    daily_location_spec,
    modal_location_from_dates_spec,
    connect,
    run_query,
    get_result_by_query_id,
)
from flowclient.aggregates import flows_spec
from pathlib import Path

In [2]:
# Default parameter values
flowapi_url = "http://localhost:9090"
aggregation_unit = "admin2"
date_ranges = [
    ["2016-01-01", "2016-01-03"],
    ["2016-01-03", "2016-01-04"],
    ["2016-01-04", "2016-01-05"],
]

In [4]:
# Connect to FlowAPI
conn = connect(
    url=flowapi_url,
    token=environ["FLOWAPI_TOKEN"],
    ssl_certificate=True,
)

/home/john/.local/lib/python3.8/site-packages/flowclient/client.py:46: UserWarning: Communications with this server are NOT SECURE.
  return Connection(


In [ ]:
print("This should be redacted")
print(environ["FLOWAPI_TOKEN"])

In [5]:
# Define date periods of interest
periods_of_interest = dict(
    benchmark=date_ranges[0], comparison=date_ranges[1], focal=date_ranges[2]
)

In [6]:
# Modal location query specifications
locations = {}
for key, period in periods_of_interest.items():
    query_spec = modal_location_from_dates_spec(
        start_date=period[0],
        end_date=period[1],
        method="last",
        aggregation_unit=aggregation_unit,
    )
    locations[key] = query_spec

In [7]:
# Flow specifications
flownormal = flows_spec(
    from_location=locations["benchmark"],
    to_location=locations["comparison"],
)
flowabnormal = flows_spec(
    from_location=locations["benchmark"],
    to_location=locations["focal"],
)

In [8]:
# Run flows
res_id_flownormal = run_query(connection=conn, query_spec=flownormal)
res_id_flowabnormal = run_query(connection=conn, query_spec=flowabnormal)

In [9]:
# Get results
get_result_by_query_id(connection=conn, query_id=res_id_flownormal)

TypeError: request() got an unexpected keyword argument 'follow_redirects'

In [10]:
get_result_by_query_id(connection=conn, query_id=res_id_flowabnormal)

TypeError: request() got an unexpected keyword argument 'follow_redirects'

In [ ]:
# Glue query IDs and date periods so that they can be accessed in other notebooks
sb.glue(
    "query_ids", {"flownormal": res_id_flownormal, "flowabnormal": res_id_flowabnormal}
)
sb.glue("date_periods", periods_of_interest)